In [ ]:
import sys; sys.path.append("../resources/")
from dotplot_utils import *
import matplotlib
import pandas as pd
import seaborn as sns
# make this notebook work better with Scanpy
import warnings; warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# make output directories
import os
if not os.path.exists("scRNA_out"):
    os.mkdir("scRNA_out")

if not os.path.exists("scRNA_out/plots_overview/"):
    os.mkdir("scRNA_out/plots_overview/")

In [ ]:
sc.set_figure_params(transparent=True, dpi_save=400)
sns.set_style("white")
sc.settings.figdir = "scRNA_out/plots_overview/"

---
## Read in key dataframe with sample information

In [ ]:
sample_key = pd.read_csv("../resources/ST/visium_sample_key.csv", index_col=0)

---
## Define gene signatures

In [ ]:
# recombine signatures into useful categories
signatures_epi_stroma = [
    'IFN stimulated EPI',
    'iCMS2',
    'iCMS3',
    'Basal',
    'Squamous',
    'Mesenchymal',
    'Glandular',
    'Ciliated',
    'Alveolar',
    'AC', # astrocyte
    'OPC', # oligodendrocyte progenitor
    'NPC', # neural progenitor
    'IES', # custom epithelial-intrinsic exclusion signature
]

signatures_myeloid = [
    'Myeloid',
    'cDC2',
    'cDC1',
    'Macrophage',
    'Classical monocytes',
    'M1',
    'M2',
    'Neutrophils',
    'Costimulatory MYE',
    'Stimulatory DCs',
    'IFN stimulated MYE',
]

signatures_lymphoid = [
    'T cell',
    'T reg',
    'T cell CD4',
    'T cell CD8',
    'T reg resting',
    'T reg suppressive',
    'T reg tissue homing',
    'T reg cytokines',
    'T reg activation',
    'TH1',
    'TH2',
    'TH17',
    'T cell exhaustion',
    'TRM', # tissue-resident memory T cells
    'NK cell',
    'B cell',
    'Plasma cell',
]

signatures_activity = [
    'EMT',
    'pEMT',
    'Senescence',
    'SASP',
    'Interferon',
    'Hypoxia',
    'Oxphos',
    'Stress',
    'Stress response',
    'Bacterial response',
    'Fibrosis',
    'Cytotoxicity',
    'Exhaustion',
    'Cytokines',
    'Chemokines',
    'MHC',
    'Fetal',
    'Stem',
    'Metaplasia',
    'Proliferation',
    'Translation',
    'Cycle',
    'Metal',
    'CytoTRACE',
    'CNV score',
    'TMB',
]

signatures_curated = [
    # activity
    'pEMT',
    'Metaplasia',
    'Stem',
    'CytoTRACE',
    'IES',
    'CNV score',
    'MHC',
    'Fibrosis',
    'Bacterial response',
    # epi/stroma
    'IFN stimulated EPI',
    'iCMS2',
    'iCMS3',
    'Squamous',
    # lymphoid
    'T reg suppressive',
    'T cell CD4',
    'T cell CD8',
    'T cell exhaustion',
    # myeloid
    'IFN stimulated MYE',
    'Neutrophils',
]

signatures_fig_1 = [
    "Stem",
    "T cell CD4",
    "iCMS2",
    "iCMS3",
    "Metaplasia",
    "CytoTRACE",
    "CNV score",
    "T cell CD8"
]

signatures_fig_2 = [
    "Stem",
    "T cell CD4",
    "iCMS2",
    "iCMS3",
    "Metaplasia",
    "CytoTRACE",
    "CNV score",
    "TMB",
    "T cell CD8",
    "IFN stimulated MYE",
    "T reg suppressive",
    "T cell exhaustion",
]

---
## Read in samples

In [ ]:
nl_epi = sc.read("../data/scRNA/VUMC_HTAN_DIS_EPI_V2.h5ad")
epi = sc.read("../data/scRNA/abnormal_epithelium.h5ad")
stroma = sc.read("../data/scRNA/VUMC_HTAN_VAL_DIS_NONEPI_V2.h5ad")

## Dotplot with signatures

In [ ]:
cmap_dict = {
    # Tumor Type
    'SSL/HP':"#c4a4e1",'MSI-H':"#7a4fa3",'MSS':"#ffc101",'TA/TVA':"#fee799",'NL':"#1f77b4",
    # Tumor Location
    "Cecum":"#62a32e","Ascending":"#b0dc7d","Hepatic Flexure":"#ebf6db","Transverse":"#fbe6f1","Descending":"#eeadd4","Sigmoid":"#cf4191",
    # this one's global
    "nan":"#ffffff",
    # These are black and white for T and F
    "T":"#000000","F":"#ffffff",
}
stage_colordict = dict(zip(["AD","I","II","III/IV"], sns.color_palette("Reds", len(["AD","I","II","III/IV"])).as_hex()))
grade_colordict = dict(zip(["G1","G2","G3"], sns.color_palette("Reds", len(["G1","G2","G3"])).as_hex()))
cmap_dict = {**cmap_dict, **stage_colordict, **grade_colordict}

In [ ]:
patient_colordict = dict(zip(sample_key.patient_name, [cmap_dict[x] for x in sample_key.tumor_type]))

In [ ]:
signatures_activity_height = 7
signatures_epi_stroma_height = 4.1
signatures_myeloid_height = 3
signatures_lymphoid_height = 4.7
signatures_curated_height = 6
signatures_fig_1_height = 3
refNMF_height = 8
archetypes_height = 2.7
mxif_height = 8.2

In [ ]:
# define heatmap widths
milwrm_width = 4.1
tumor_type_width = 3.2
tumor_loc_width = 3.7
tumor_stage_width = 3.2
tumor_grade_width = 3.0
patient_width = 4.9
pathology_width = 4.0

In [ ]:
# concatenate all tumor compartments
a = epi.concatenate([nl_epi, stroma], fill_value=0); a

In [ ]:
a.obs["Tumor_Type"].replace({"MSI":"MSI-H"}, inplace=True)

In [ ]:
# read in mutational burden per sample from WES data outputs
variants = pd.read_csv("../step1/WES_out/pseudobulk_variants_per_sample.csv", index_col=0)

In [ ]:
# consolidate spec IDs for MAP8622 polyps
a.obs.loc[a.obs.Patient=="HTA11_08622","HTAN Specimen ID"].value_counts()
a.obs.loc[a.obs["HTAN Specimen ID"].isin(["HTA11_8622_2000001011","HTA11_8622_2000001021","MAP08622_A"]), "Patient"] = "HTA11_08622_A"
a.obs.loc[a.obs["HTAN Specimen ID"].isin(["HTA11_8622_2000002011","HTA11_8622_2000002021","MAP08622_B"]), "Patient"] = "HTA11_08622_B"

In [ ]:
a.obs.Patient.value_counts()

In [ ]:
a.obs["Variants"] = 0
for s in variants.Tumor_Sample_Barcode:
    try:
        a.obs.loc[a.obs.Patient == s, "Variants"] = variants.loc[variants.Tumor_Sample_Barcode==s, "Variants"].values[0]
    except:
        print("No bulk variants found")

In [ ]:
# rename 'Variants' to 'TMB' (Tumor Mutational Burden)
a.obs.rename(columns={"Variants":"TMB"}, inplace=True)

---
# Overview plots

In [ ]:
for features_name, features_list, height in zip(
        ["signatures_activity", "signatures_epi_stroma", "signatures_myeloid", "signatures_curated", "signatures_fig_1"],
        [signatures_activity, signatures_epi_stroma, signatures_myeloid, signatures_curated, signatures_fig_1],
        [signatures_activity_height, signatures_epi_stroma_height, signatures_myeloid_height, signatures_curated_height, signatures_fig_1_height],
    ):
    # set counter to 0 for the first group
    counter = 0
    for group_name, group, width, groupby_order, groupby_colordict, in zip(
        ["tumortype", "patient"],
        ["Tumor_Type", "Patient"],
        [tumor_type_width, patient_width],
        [None, None],
        [cmap_dict, patient_colordict],
    ):
        if counter == 0:
            # for the first group, hierarchically cluster the features
            features_ordered = cody_heatmap(
                a,
                groupby=group,
                features=features_list,
                cluster_vars=True,
                vars_dict=None,
                groupby_order=groupby_order,
                groupby_colordict=groupby_colordict,
                cluster_obs=True if groupby_order is None else False,
                figsize=(width, height),
                save="scRNA_out/plots_overview/{}_{}_dotplot.png".format(group_name, features_name),
                dpi=400,
                cmap="Greys",
                size_title="Fraction of cells or\nspots in group (%)",
            )
            counter += 1
        else:
            # after the first group, features are ordered based on the first group clustering
            cody_heatmap(
                a,
                groupby=group,
                features=features_ordered,
                cluster_vars=False,
                vars_dict=None,
                groupby_order=groupby_order,
                groupby_colordict=groupby_colordict,
                cluster_obs=True if groupby_order is None else False,
                figsize=(width, height),
                save="scRNA_out/plots_overview/{}_{}_dotplot.png".format(group_name, features_name),
                dpi=400,
                cmap="Greys",
                size_title="Fraction of cells or\nspots in group (%)",
            )

In [ ]:
# re-run curated signatures with fixed ordering
for features_name, features_list, height in zip(
        ["signatures_fig_1"],
        [signatures_fig_1],
        [2.4],
    ):
    # set counter to 0 for the first group
    counter = 0
    for group_name, group, width, groupby_order, groupby_colordict, in zip(
        ["tumortype", "patient"],
        ["Tumor_Type", "Patient"],
        [tumor_type_width, patient_width],
        [["NL","TA/TVA","MSS","SSL/HP","MSI-H"], [
                "HTA11_01938",
                "HTA11_07862",
                "HTA11_10711",
                "PAT71397",
                "PAT71662",
                "PAT73899",
                "PAT74143",
                "HTA11_06134",
                "HTA11_07663",
                "HTA11_08622_A",
                "HTA11_08622_B",
                "PAT73458",
            ]],
        [cmap_dict, patient_colordict],
    ):
        if counter == 1:
            pass
        else:
            features_ordered = [
                "Stem",
                "T cell CD4",
                "iCMS2",
                "iCMS3",
                "Metaplasia",
                "CytoTRACE",
                "CNV score",
                "T cell CD8",
            ]
            # after the first group, features are ordered based on the first group clustering
            cody_heatmap(
                a,
                groupby=group,
                features=features_ordered,
                cluster_vars=False,
                vars_dict=None,
                groupby_order=groupby_order,
                groupby_colordict=groupby_colordict,
                cluster_obs=True if groupby_order is None else False,
                figsize=(width, height),
                save="scRNA_out/plots_overview/{}_{}_dotplot.png".format(group_name, features_name),
                dpi=400,
                cmap="Greys",
                size_title="Fraction of cells or\nspots in group (%)",
            )

In [ ]:
group_name = "patient"
features_name = "signatures_fig_1"
group = "Patient"
features_list = signatures_fig_1
height = signatures_fig_1_height
width = patient_width
groupby_colordict = patient_colordict
groupby_order = [
    "HTA11_01938",
    "HTA11_07862",
    "HTA11_10711",
    "PAT71397",
    "PAT71662",
    "PAT73899",
    "PAT74143",
    "HTA11_06134",
    "HTA11_07663",
    "HTA11_08622_A",
    "HTA11_08622_B",
    "PAT73458",
]
features_ordered = [
    "Stem",
    "T cell CD4",
    "iCMS2",
    "iCMS3",
    "Metaplasia",
    "CytoTRACE",
    "CNV score",
    "T cell CD8",
]

cody_heatmap(
    a,
    groupby=group,
    features=features_ordered,
    cluster_vars=False,
    vars_dict=None,
    groupby_order=groupby_order,
    groupby_colordict=groupby_colordict,
    cluster_obs=True if groupby_order is None else False,
    figsize=(width, 2.4),
    save="scRNA_out/plots_overview/{}_{}_dotplot.png".format(group_name, features_name),
    dpi=400,
    cmap="Greys",
    size_title="Fraction of cells or\nspots in group (%)",
)

---
# Gene and cell state summaries for immune exclusion analysis

In [ ]:
custom_dict = {
    "PPT": ["CNV score","TMB"],
    "Excl.": ["IES","DDR1","TGFBI","PAK4","DPEP1","Fibrosis"],
    "uEnv.": ["CXCL14","CXCL12","SPP1"],
    "Inf.": ["T cell CD4","T reg suppressive","TRAC","T cell CD8","GZMB","GZMA","PLAUR","CD207"],
    "Act.": ["Oxphos","Hypoxia","pEMT"],
    "MSS": ["iCMS2","Stem","CEACAM6"],
    "MSI": ["iCMS3","MUC5AC","Metaplasia"],
    "NL": ["MUC2","SLC26A3","BEST4"],
}
custom_dict_height = 7.5

In [ ]:
for features_name, features_list, height in zip(
        ["immexcl"],
        [custom_dict],
        [custom_dict_height],
    ):
    for group_name, group, width, groupby_order, groupby_colordict, in zip(
        ["tumortype", "patient"],
        ["Tumor_Type", "Patient"],
        [3.8, patient_width],
        [None, None],
        [cmap_dict, patient_colordict],
    ):
        cody_heatmap(
            a,
            groupby=group,
            features=sum(custom_dict.values(), []),
            cluster_vars=False,
            vars_dict=custom_dict,
            groupby_order=groupby_order,
            groupby_colordict=groupby_colordict,
            cluster_obs=True if groupby_order is None else False,
            figsize=(width, height),
            save="scRNA_out/plots_overview/{}_{}_dotplot.png".format(group_name, features_name),
            dpi=400,
            cmap="Greys",
            size_title="Fraction of cells\nin group (%)",
        )